## Тип документа

Тип документа указывается в шапке документа (в верхней части документа).

В рамках задания документ может относиться к одному из следующих типов:

* «федеральный закон»
* «постановление»
* «приказ»
* «распоряжение»
* «закон»
* «указ»

Обратите внимание, что в перечне присутствуют два похожих, но отдельных типа: «закон» и «федеральный закон».

Поле type должно содержать строку, обозначающую какой-то один из указанных типов, это задача классификации.

**Оценка качества:** Требуется предсказать один из шести классов. Метрика качества – макро f1-мера.

Признаки:

* количество "федеральный", "закон", "постановление", "приказ", "распоряжение", "указ", "федеральный закон" в тексте;
* позиция, в которой встретилось каждое из этих слов;
* бинарный признак - если какое-то из слов встретилось и написано капсом;
* бинарный признак - если слово единственное в данной строке (учитываем слова из букв);
* бинарный признак - если слово первое в строке (документе);
* учет опечаток в словах??? - м. б. находить расстояние между словами (если слово написано капсом и одно в строке, то можно его сравнить со словами из нашего множества)
* можно добавить еще поиск слов "постановляет" и т. д.

In [1]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
import json
import re
import numpy as np

In [5]:
reg_exprs = [
    re.compile(r'[Ф|ф]едеральный [З|з]акон'),
    re.compile(r'ФЕДЕРАЛЬНЫЙ ЗАКОН'),
    re.compile(r'[П|п]остановление'),
    re.compile(r'ПОСТАНОВЛЕНИЕ'),
    re.compile(r'[З|з]акон'),
    re.compile(r'ЗАКОН'),
    re.compile(r'[П|п]риказ'),
    re.compile(r'ПРИКАЗ'),
    re.compile(r'[Р|р]аспоряжение'),
    re.compile(r'РАСПОРЯЖЕНИЕ'),
    re.compile(r'[У|у]каз'),
    re.compile(r'УКАЗ')
]

In [13]:
with open('train/txts/febb5f7a03b7f101bf4dbb644ef00f2daf935c6b.txt', 'r') as f:
    doc = f.read()
    features = np.array([len(expr.findall(doc)) for expr in reg_exprs])

In [2]:
TRAIN_DIR = 'train/txts/'

In [29]:
def make_features(X):
    """
    X - list of document names
    returns list of lists with document features
    """
    features = []
    for doc_name in X:
        with open(TRAIN_DIR + doc_name + '.txt', 'r') as f:
            doc = f.read()
        doc_features = np.array([len(expr.findall(doc)) for expr in reg_exprs])
        features.append(doc_features)
    return np.array(features)

In [30]:
classes2num = {
    'федеральный закон': 0,
    'постановление': 1,
    'приказ': 2,
    'распоряжение': 3,
    'закон': 4,
    'указ': 5
}

In [35]:
names, y = [], []
with open("train/gold_labels.txt", "r") as read_file:
    for doc_info in read_file.readlines():
        doc_dict = json.loads(doc_info)
        names.append(doc_dict['id'])
        y.append(classes2num[doc_dict['label']['type']])
names, y = np.array(names), np.array(y)

In [36]:
features_extractor = MakeFeatures() 
X = features_extractor.transform(names)

In [42]:
clf = XGBClassifier()
scores = cross_val_score(clf, X, y, cv=5, scoring='f1_macro')

In [43]:
scores

array([0.98918963, 0.99118965, 0.98150286, 0.9947623 , 0.97904905])

In [3]:
train = []
with open("train/gold_labels.txt", "r") as read_file:
    for doc_info in read_file.readlines():
        doc_dict = json.loads(doc_info)
        docname = TRAIN_DIR + doc_dict['id'] + '.txt'
        with open(docname, 'r') as f:
            train.append((f.read(), doc_dict['label']))

In [4]:
from task1 import Solution
from sklearn.model_selection import train_test_split
from eval_module import quality

train_data, test = train_test_split(train)
solution = Solution()
solution.train(train_data)
test_data = [x[0] for x in test]
test_labels = [x[1] for x in test]
result = solution.predict(test_data)
quality(result, test_labels)

{'date_accuracy': 0.0,
 'number_accuracy': 0.0,
 'type_f1_score': 0.9838392278,
 'name_jaccard': 0.0,
 'authority_jaccard': 0.0,
 'subtasks_improves': 1}